In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = '/mnt/workspace/chatglm3-6b'

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.57it/s]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

2024-02-08 01:37:32.429318: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-08 01:37:33.339538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
print(response)

乾卦是《易经》中的一个卦象，它是由两个阴爻夹一个阳爻构成，象征着天、干燥、强盛、积极、刚健等特质。乾卦的卦辞为：“元亨，利见大人。”意味着这个卦象具有向上的能量和创造力，预示着事物的发展将充满活力和希望。

乾卦的六爻分别代表不同的阶段和含义。从下至上，分别为：初爻、二爻、三爻、四爻、五爻和上爻。每一爻的阳爻代表阳刚之气，阴爻代表阴柔之气。通过六爻的组合，乾卦展示了事物发展变化的规律。

乾卦的动态变化象征着阳刚之气和阴柔之气的相互作用，提醒人们在面对事物时要保持中庸之道，即不要过于刚烈，也不要过于柔弱。同时，乾卦还告诉我们，事物的发展是不断变化的，要善于适应和调整，才能达到和谐的状态。


In [8]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中的一个卦象，由两个阴爻夹一个阳爻构成，象征着地、水、智、勇等特质。它的卦辞为：“元、亨，利、利见大人。”意味着这个卦象具有向下的能量和创造力，预示着事物的发展将充满智慧和希望。

地水师卦的六爻分别代表不同的阶段和含义。从下至上，分别为：初爻、二爻、三爻、四爻、五爻和上爻。每一爻的阳爻代表阳刚之气，阴爻代表阴柔之气。通过六爻的组合，地水师卦展示了事物发展变化的规律。

地水师卦的动态变化象征着阳刚之气和阴柔之气的相互作用，提醒人们在面对事物时要保持中庸之道，即不要过于刚烈，也不要过于柔弱。同时，地水师卦还告诉我们，事物的发展是不断变化的，要善于适应和调整，才能达到和谐的状态。


## 微调后

In [9]:
from peft import PeftModel, PeftConfig

peft_model_path = f"/mnt/workspace/models/chatglm3-6b-epoch3-20240207_165140"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [10]:
inputs = tokenizer(input_text, return_tensors="pt").to(0)
ft_out = model.generate(**inputs, max_new_tokens=512)
ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)

In [11]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [12]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由两个乾卦叠加而成，代表刚健、健行、刚健不屈的意境。在周易中，它是阳卦，代表君子品性，也象征着君子行事刚健、正直、勇敢。乾卦的卦象是由六个阳爻组成的，它代表天宇广阔，掌握着宇宙间的一切。乾卦的核心哲学是：君子观此卦象，从而以刚健、健行、不屈的精神行事。

乾卦的核心哲学在于天行健，君子观此卦象，从而以天为榜样，实行刚健、健行、不屈的精神。在事业上，乾卦提示着行事刚健、正直、勇敢，不屈不挠，勇于接受挑战，得险为夷。在经商中，乾卦提醒着实施计划，坚定不移，勇于承担风险，刚健不屈，从而获得成功。在感情方面，乾卦强调刚健、忠诚、正直，交往中要求刚健、忠诚、正直，从而获得他人的尊敬和信任。

总体而言，乾卦是一个正面的卦象，它提示着君子行事刚健、正直、勇敢，不屈不挠，得险为夷，得志忘形。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [13]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是一个由卦象组成的卦名，在周易中，它是由两个卦相组合而成，即坤卦（地）和坎卦（水）。这一卦的主要含义是：柔顺地，顺和地，地 nao 卦。根据《易经》，这一卦代表着预测吉凶可卜。地水师卦象征着地中有水，地保存着水分，预示着君子应当适应环境，顺应自然，掌握技巧，必然吉顺。

地水师卦的形成原因是：地雷卦（雷卦）下，坎卦上。雷卦象征着雷电，代表着力量和刚毅，而坎卦代表着智慧、预测。在这卦中，地象征大吉，雷象征吉祥，而卦象组合 indicative of大吉。因此，地水师卦整体上象征着大吉，但需谨慎行事，顺应自然，方能取得吉祥。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 您说的地水师卦可能是指“地水师卦”，其由卦象组合而成。在这个卦中，下卦是乾卦，上卦是坤卦。乾卦代表天，象征刚健，坤卦则代表地，象征顺从。组合起来，代表地天相融，形成一种和谐的氛围。地水师卦象征着吉祥，预示着吉祥如意，有吉祥之象。

地水师卦的卦辞这样描述：“天动于地，名曰地水师卦。吉，吉利如意，有吉祥之象。梗塞之疾，忧愁之极，莫测其忧，必须坚忍，方能成功。宜坚忍，忍耐，宜速行。 "

地水师卦是一个吉祥的卦象，但也要根据实际情况和努力方向来判断。如果求得吉祥如意的结果，需要坚忍忍耐，如果能够妥善处理事务，那么便能够获得吉利。


In [14]:
! nvidia-smi

Thu Feb  8 01:41:44 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:08.0 Off |                    0 |
| N/A   36C    P0    55W / 300W |   5748MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------